In [1]:
import tkinter as tk
from tkinter import Scrollbar, Text, filedialog, messagebox
import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import sqlite3
import spacy
import subprocess
import sys
import threading
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


# ==================================================
# Configuración del Chatbot
# ==================================================

# Función para descargar el modelo si no está disponible
def load_spacy_model():
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
        return spacy.load("en_core_web_sm")

# Cargar modelo de NLP
nlp = load_spacy_model()

# Crear base de datos con preguntas y respuestas predefinidas
conn = sqlite3.connect("alzheimers_chatbot.db")
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS faq (question TEXT, answer TEXT)''')

# Preguntas y respuestas predefinidas
faq_data = [
    ("¿Qué es el Alzheimer?", "El Alzheimer es una enfermedad neurodegenerativa que afecta la memoria y otras funciones cognitivas."),
    ("¿Cuáles son los síntomas del Alzheimer?", "Los síntomas incluyen pérdida de memoria, confusión, dificultad para resolver problemas y cambios de humor."),
    ("¿Existe cura para el Alzheimer?", "Actualmente no hay cura, pero existen tratamientos para ralentizar el progreso de la enfermedad."),
    ("¿Cuáles son los factores de riesgo?", "Los principales factores de riesgo incluyen la edad, antecedentes familiares y estilo de vida poco saludable."),
    ("Hola!", "¡Hola! ¿En qué puedo ayudarte hoy?"),
    ("¿Qué puedes hacer?", "Puedo responder preguntas sobre el Alzheimer, sus síntomas, cuidados y tratamientos."),
    ("Mi familiar tuvo una pérdida de memoria, ¿qué hago?", "Si notas pérdida de memoria en un familiar, es recomendable acudir a un especialista para una evaluación médica."),
    ("¿Cómo prevenir el Alzheimer?", "Mantener una dieta saludable, hacer ejercicio regularmente y mantener la mente activa pueden ayudar a reducir el riesgo."),
    ("¿Cuáles son los tratamientos actuales?", "Existen medicamentos que pueden ayudar a ralentizar los síntomas, pero no curan la enfermedad."),
    ("¿El Alzheimer es hereditario?", "El riesgo de desarrollar Alzheimer puede aumentar si hay antecedentes familiares, pero no siempre es hereditario."),
    ("¿Qué hacer si un paciente con Alzheimer se pierde?", "Es importante mantener identificación en el paciente y avisar a las autoridades en caso de extravío."),
    ("¿Cómo puedo comunicarme mejor con alguien con Alzheimer?", "Habla con frases simples, mantén contacto visual y usa un tono calmado y amable."),
    ("¿Cuánto tiempo vive una persona con Alzheimer?", "El tiempo de vida varía, pero en promedio es de 4 a 8 años después del diagnóstico."),
    ("¿Cómo manejar el estrés de cuidar a un paciente con Alzheimer?", "Descansa, busca apoyo en grupos de ayuda y organiza tu tiempo para evitar el agotamiento."),
]

# Insertar datos en la base de datos (si no existen)
c.executemany("INSERT OR IGNORE INTO faq (question, answer) VALUES (?, ?)", faq_data)
conn.commit()

# Crear un vectorizador TF-IDF
questions = [q for q, a in faq_data]
vectorizer = TfidfVectorizer().fit(questions)

# Función para obtener respuesta del chatbot usando similitud del coseno
def get_response(user_input):
    user_input = user_input.lower()
    
    # Transformar la entrada del usuario y las preguntas predefinidas a vectores TF-IDF
    user_vector = vectorizer.transform([user_input])
    question_vectors = vectorizer.transform(questions)
    
    # Calcular la similitud del coseno entre la entrada del usuario y las preguntas predefinidas
    similarities = cosine_similarity(user_vector, question_vectors)
    
    # Obtener el índice de la pregunta más similar
    most_similar_index = similarities.argmax()
    
    # Obtener la similitud más alta
    highest_similarity = similarities[0, most_similar_index]
    
    # Si la similitud es mayor que un umbral, devolver la respuesta correspondiente
    if highest_similarity > 0.5:  # Umbral de similitud
        return faq_data[most_similar_index][1]
    
    # Análisis con NLP para mejorar respuestas
    doc = nlp(user_input)
    for keyword in ["síntomas", "factores", "tratamiento", "diagnóstico", "prevención", "memoria", "cuidado", "comunicación", "estrés"]:
        if keyword in user_input:
            return "Parece que preguntas sobre " + keyword + ". ¿Podrías ser más específico?"
    
    return "Lo siento, no tengo una respuesta para eso. Intenta reformular tu pregunta."

# ==================================================
# Configuración del Monitoreo
# ==================================================

# Función para simular datos de sensores
def simular_datos_sensores():
    movimiento = np.random.rand()
    sueño = np.random.rand()
    habla = np.random.rand()
    ubicacion = np.random.choice(["Casa", "Fuera de casa"])
    frecuencia_cardiaca = np.random.randint(60, 100)
    estado_animo = np.random.choice(["Tranquilo", "Agitado", "Deprimido"])
    medicamentos = np.random.choice(["Tomado", "No tomado"])
    presion_arterial = f"{np.random.randint(90, 140)}/{np.random.randint(60, 90)}"
    glucosa = np.random.randint(70, 140)
    temperatura = np.random.uniform(36.0, 38.0)
    hidratacion = np.random.rand()
    actividad_social = np.random.choice(["Activo", "Aislado"])
    ingesta_alimentos = np.random.choice(["Adecuada", "Inadecuada"])
    entorno_temperatura = np.random.uniform(18.0, 30.0)
    entorno_humedad = np.random.uniform(30.0, 70.0)
    return movimiento, sueño, habla, ubicacion, frecuencia_cardiaca, estado_animo, medicamentos, presion_arterial, glucosa, temperatura, hidratacion, actividad_social, ingesta_alimentos, entorno_temperatura, entorno_humedad

# Función para generar alertas basadas en los datos de sensores
def generar_alertas(movimiento, sueño, habla, ubicacion, frecuencia_cardiaca, estado_animo, medicamentos, presion_arterial, glucosa, temperatura, hidratacion, actividad_social, ingesta_alimentos, entorno_temperatura, entorno_humedad):
    alertas = []
    if movimiento < 0.1:
        alertas.append("Alerta: Baja actividad física detectada.")
    if sueño < 0.2:
        alertas.append("Alerta: Sueño de mala calidad detectado.")
    if habla < 0.3:
        alertas.append("Alerta: Cambios en el habla detectados.")
    if ubicacion == "Fuera de casa":
        alertas.append("Alerta: El paciente ha salido de casa.")
    if frecuencia_cardiaca < 60 or frecuencia_cardiaca > 100:
        alertas.append(f"Alerta: Frecuencia cardíaca anormal detectada ({frecuencia_cardiaca} lpm).")
    if estado_animo == "Agitado":
        alertas.append("Alerta: El paciente está agitado.")
    if estado_animo == "Deprimido":
        alertas.append("Alerta: El paciente está deprimido.")
    if medicamentos == "No tomado":
        alertas.append("Alerta: El paciente no ha tomado sus medicamentos.")
    if int(presion_arterial.split('/')[0]) > 140 or int(presion_arterial.split('/')[1]) > 90:
        alertas.append(f"Alerta: Presión arterial alta detectada ({presion_arterial} mmHg).")
    if int(presion_arterial.split('/')[0]) < 90 or int(presion_arterial.split('/')[1]) < 60:
        alertas.append(f"Alerta: Presión arterial baja detectada ({presion_arterial} mmHg).")
    if glucosa < 70:
        alertas.append(f"Alerta: Hipoglucemia detectada ({glucosa} mg/dL).")
    if glucosa > 140:
        alertas.append(f"Alerta: Hiperglucemia detectada ({glucosa} mg/dL).")
    if temperatura > 37.5:
        alertas.append(f"Alerta: Fiebre detectada ({temperatura:.1f} °C).")
    if temperatura < 36.0:
        alertas.append(f"Alerta: Hipotermia detectada ({temperatura:.1f} °C).")
    if hidratacion < 0.3:
        alertas.append("Alerta: Deshidratación detectada.")
    if actividad_social == "Aislado":
        alertas.append("Alerta: El paciente está aislado socialmente.")
    if ingesta_alimentos == "Inadecuada":
        alertas.append("Alerta: Ingesta inadecuada de alimentos detectada.")
    if entorno_temperatura < 20.0 or entorno_temperatura > 25.0:
        alertas.append(f"Alerta: Temperatura ambiente inadecuada ({entorno_temperatura:.1f} °C).")
    if entorno_humedad < 40.0 or entorno_humedad > 60.0:
        alertas.append(f"Alerta: Humedad ambiente inadecuada ({entorno_humedad:.1f}%).")
    return alertas

# Variable para controlar el monitoreo
monitoreo_activo = True

# Función para monitorear comportamiento en tiempo real
def monitorear_comportamiento():
    while not detener_monitoreo.is_set():
        movimiento, sueño, habla, ubicacion, frecuencia_cardiaca, estado_animo, medicamentos, presion_arterial, glucosa, temperatura, hidratacion, actividad_social, ingesta_alimentos, entorno_temperatura, entorno_humedad = simular_datos_sensores()
        alertas = generar_alertas(movimiento, sueño, habla, ubicacion, frecuencia_cardiaca, estado_animo, medicamentos, presion_arterial, glucosa, temperatura, hidratacion, actividad_social, ingesta_alimentos, entorno_temperatura, entorno_humedad)
        for alerta in alertas:
            alertas_text.insert(tk.END, f"[{datetime.now()}] {alerta}\n")
            alertas_text.see(tk.END)  # Desplazar automáticamente hacia abajo
        time.sleep(5)

# Función para detener el monitoreo
def detener_monitoreo():
    global monitoreo_activo
    monitoreo_activo = False
    detener_monitoreo_event.set()  # Señalar al hilo que debe detenerse
    alertas_text.insert(tk.END, f"[{datetime.now()}] Monitoreo desactivado.\n")
    alertas_text.see(tk.END)

# Función para continuar el monitoreo
def continuar_monitoreo():
    global monitoreo_activo
    monitoreo_activo = True
    detener_monitoreo_event.clear()  # Limpiar el evento para permitir que el hilo continúe
    alertas_text.insert(tk.END, f"[{datetime.now()}] Monitoreo activado.\n")
    alertas_text.see(tk.END)
    monitoreo_thread = threading.Thread(target=monitorear_comportamiento, daemon=True)
    monitoreo_thread.start()

# Función para predecir
model = load_model('mejor_modelo.h5')

# Función para abrir el explorador de archivos y seleccionar una imagen
def seleccionar_imagen():
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal de tkinter
    file_path = filedialog.askopenfilename(
        title="Selecciona una imagen",
        filetypes=(("Archivos de imagen", "*.jpg *.jpeg *.png"), ("Todos los archivos", "*.*"))
    )
    return file_path

# Función para preprocesar la imagen y hacer la predicción
def predecir_imagen(file_path):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(150, 150))
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediccion = model.predict(img)
    clase_predicha = np.argmax(prediccion, axis=1)
    return clase_predicha[0]


# Función para manejar la selección de imagen y mostrar la predicción
def predecir_y_mostrar():
    file_path = seleccionar_imagen()
    if file_path:
        clase_predicha = predecir_imagen(file_path)
        # Aquí puedes personalizar el mensaje según la clase predicha
        if clase_predicha == 0:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa leve del alzheimer.\n Se sugiere que opte por participar en actividades cognitivas y físicas para estimular el cerebro.\n Consultar regularmente in médico especializado en neurología.\n  Se observa una disminución en el volumen del tejido cerebral, especialmente en áreas como el hipocampo (crucial para la memoria) y la corteza cerebral. Esta atrofia es un indicador clave de la enfermedad de Alzheimer"
        elif clase_predicha == 1:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa avanzada del alzheimer. \n Se observa una atrofia cerebral más pronunciada en comparación con la resonancia magnética.\n Los ventrículos están más agrandados y la corteza cerebral muestra una mayor pérdida de volumen.\n Se recomienda evaluaciones médicas de forma frecuente"
        elif clase_predicha == 2:
            resultado = "La imagen proporcionada pertenece a una persona que no posee alzheimer. \n  En general, la estructura cerebral parece estar dentro de los límites normales para la edad de la persona. \n No se observa una atrofia cerebral significativa. El volumen del tejido cerebral parece conservado."
        elif clase_predicha == 3:
            resultado = "La imagen proporcionada pertenece a una persona que puede tener indicios de padecer alzheimer. \n Se observa que el patrón de atrofia es consistente con la enfermedad de Alzheimer en una etapa muy temprana. Por lo tanto, existe una posibilidad de que esta persona padezca de alzheimer "
        else:
            resultado = "La imagen proporcionada es incorrecta."
        resultado_text.delete(1.0, tk.END)  # Limpiar el texto anterior
        resultado_text.insert(tk.END, resultado)

def cerrar_ventana():
    detener_monitoreo()
    ventana.destroy()

# ==================================================
# Interfaz gráfica
# ==================================================

# Crear la ventana principal
ventana = tk.Tk()
ventana.title("Sistema de Monitoreo y Chatbot para Alzheimer")
ventana.geometry("1000x700")

# Sección de Monitoreo
monitoreo_frame = tk.LabelFrame(ventana, text="Monitoreo en Tiempo Real", padx=10, pady=10)
monitoreo_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

alertas_text = tk.Text(monitoreo_frame, height=15, width=80)
alertas_text.pack(fill=tk.BOTH, expand=True)

# Botones para detener y continuar el monitoreo
botones_frame = tk.Frame(monitoreo_frame)
botones_frame.pack(fill=tk.X, pady=5)

detener_button = tk.Button(botones_frame, text="Detener Monitoreo", command=detener_monitoreo)
detener_button.pack(side=tk.LEFT, padx=5)

continuar_button = tk.Button(botones_frame, text="Continuar Monitoreo", command=continuar_monitoreo)
continuar_button.pack(side=tk.LEFT, padx=5)


# Sección de Chatbot
chatbot_frame = tk.LabelFrame(ventana, text="Chatbot de Alzheimer", padx=10, pady=10)
chatbot_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

scrollbar = Scrollbar(chatbot_frame)
chatbox = Text(chatbot_frame, height=15, width=80, yscrollcommand=scrollbar.set)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
chatbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

entry = tk.Entry(chatbot_frame, width=70)
entry.pack()

# Función para enviar mensaje
def send_message():
    user_text = entry.get()
    if user_text:
        chatbox.insert(tk.END, "Tú: " + user_text + "\n")
        response = get_response(user_text)
        chatbox.insert(tk.END, "Bot: " + response + "\n\n")
        entry.delete(0, tk.END)

# Botón para enviar mensaje
send_button = tk.Button(chatbot_frame, text="Enviar", command=send_message)
send_button.pack()

# Enviar mensaje al presionar Enter
entry.bind("<Return>", lambda event: send_message())

# Sección de Predicción de Imagen
prediccion_frame = tk.LabelFrame(ventana, text="Predicción de Etapa de Alzheimer", padx=10, pady=10)
prediccion_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

# Botón para seleccionar imagen y predecir
predecir_button = tk.Button(prediccion_frame, text="Seleccionar Imagen y Predecir", command=predecir_y_mostrar)
predecir_button.pack(pady=5)

# Caja de texto para mostrar el resultado de la predicción
resultado_text = tk.Text(prediccion_frame, height=5, width=80)
resultado_text.pack(fill=tk.BOTH, expand=True)

# Iniciar el monitoreo en tiempo real en un hilo separado
monitoreo_thread = threading.Thread(target=monitorear_comportamiento, daemon=True)
monitoreo_thread.start()
detener_monitoreo_event = threading.Event()
ventana.protocol("WM_DELETE_WINDOW", cerrar_ventana)
# Ejecutar el bucle principal de la aplicación
ventana.mainloop()

conn.close()

# Cerrar base de datos al salir

Exception in thread Thread-3 (monitorear_comportamiento):
Traceback (most recent call last):
  File "C:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\pablo\Desktop\Entorno2\dios\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\pablo\AppData\Local\Temp\ipykernel_10588\829768786.py", line 165, in monitorear_comportamiento
AttributeError: 'function' object has no attribute 'is_set'


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


KeyboardInterrupt: 

In [ ]:
# 